In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch

In [2]:
df = pd.read_csv('Datasets\Microsoft_Azure_Predictive_Maintenance\PdM_telemetry.csv')
df.head()

,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-01 06:00:00,1,176.217853,418.504078,113.077935,45.087686
1,2015-01-01 07:00:00,1,162.879223,402.747490,95.460525,43.413973
2,2015-01-01 08:00:00,1,170.989902,527.349825,75.237905,34.178847
3,2015-01-01 09:00:00,1,162.462833,346.149335,109.248561,41.122144
4,2015-01-01 10:00:00,1,157.610021,435.376873,111.886648,25.990511


importing models

In [8]:
models_df = pd.read_csv('Datasets\Microsoft_Azure_Predictive_Maintenance\PdM_machines.csv')
models = models_df['model'].unique()
models

array(['model3', 'model4', 'model2', 'model1'], dtype=object)

dividing the dataset based on models

In [22]:
IDs = {}
for model in models:
    m = models_df[models_df['model']==model]
    IDs[model] = list(m['machineID'])

failures

In [24]:
failure = pd.read_csv('Datasets\Microsoft_Azure_Predictive_Maintenance\PdM_failures.csv')
failure

,datetime,machineID,failure
0,2015-01-05 06:00:00,1,comp4
1,2015-03-06 06:00:00,1,comp1
2,2015-04-20 06:00:00,1,comp2
3,2015-06-19 06:00:00,1,comp4
4,2015-09-02 06:00:00,1,comp4
...,...,...,...
756,2015-11-29 06:00:00,99,comp3
757,2015-12-14 06:00:00,99,comp4
758,2015-02-12 06:00:00,100,comp1
759,2015-09-10 06:00:00,100,comp1


In [34]:
def failure_extraction(model):    
    data = []
    label = []
    ID = IDs[model]
    for i in ID:
        machine = df[df['machineID']==i]
        failure_machine = failure[failure['machineID']==i]
        machine.drop(columns=['machineID'], inplace=True)
        for timestamp in failure_machine['datetime']:
            filtered_time = machine[machine['datetime']<=timestamp]
            machine.drop(machine[machine['datetime']<=timestamp].index, inplace=True)
            filtered_time.drop(columns=['datetime'], inplace=True)
            filtered = filtered_time[-50:]
            if filtered.shape[0] == 50:
                np_array = filtered.to_numpy()
                tensor = torch.tensor(np_array, dtype=torch.float).unsqueeze(0)
                data.append(tensor)
                label.append(torch.tensor([1]))
            filtered_time.drop(filtered.index)

            filtered = filtered_time[-50:]
            # for limiting the number of non failing componnents
            j = 0
            while filtered.shape[0] == 50 and j < 5:
                np_array = filtered.to_numpy()
                tensor = torch.tensor(np_array, dtype=torch.float).unsqueeze(0)
                data.append(tensor)
                label.append(torch.tensor([0]))
                filtered_time.drop(filtered.index)
                filtered = filtered_time[-50:]
                j += 1

        data_tensor = torch.cat(data, dim=0)
        label_tensor = torch.cat(label, dim=0)
    return data_tensor, label_tensor


In [41]:
data_1, label_1 = failure_extraction(model= 'model1')
data_2, label_2 = failure_extraction(model = 'model2')
data_3, label_3 = failure_extraction(model= 'model3')
data_4, label_4 = failure_extraction(model = 'model4')
print(label_1.size())
print(label_2.size())
print(label_3.size())
print(label_4.size())

C:\Users\Asus\AppData\Local\Temp\ipykernel_32276\3650296098.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  machine.drop(columns=['machineID'], inplace=True)
C:\Users\Asus\AppData\Local\Temp\ipykernel_32276\3650296098.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  machine.drop(machine[machine['datetime']<=timestamp].index, inplace=True)
C:\Users\Asus\AppData\Local\Temp\ipykernel_32276\3650296098.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-

torch.Size([1002])
torch.Size([936])
torch.Size([1242])
torch.Size([1014])


C:\Users\Asus\AppData\Local\Temp\ipykernel_32276\3650296098.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  machine.drop(machine[machine['datetime']<=timestamp].index, inplace=True)
C:\Users\Asus\AppData\Local\Temp\ipykernel_32276\3650296098.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_time.drop(columns=['datetime'], inplace=True)
C:\Users\Asus\AppData\Local\Temp\ipykernel_32276\3650296098.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#retu

In [43]:
torch.save(data_1, 'Datasets\Microsoft_Azure_Predictive_Maintenance\data_1.pt')
torch.save(label_1, 'Datasets\Microsoft_Azure_Predictive_Maintenance\label_1.pt')
torch.save(data_2, 'Datasets\Microsoft_Azure_Predictive_Maintenance\data_2.pt')
torch.save(label_2, 'Datasets\Microsoft_Azure_Predictive_Maintenance\label_2.pt')
torch.save(data_3, 'Datasets\Microsoft_Azure_Predictive_Maintenance\data_3.pt')
torch.save(label_3, 'Datasets\Microsoft_Azure_Predictive_Maintenance\label_3.pt')
torch.save(data_4, 'Datasets\Microsoft_Azure_Predictive_Maintenance\data_4.pt')
torch.save(label_4, 'Datasets\Microsoft_Azure_Predictive_Maintenance\label_4.pt')